<a href="https://colab.research.google.com/github/HwangHanJae/recommender_system/blob/main/inflearn_recsys/CF_improve_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 그 외의 정확도 개선 방법

공통 평가한 아이템이 2개인 사용자와 10개인 사용자가 동일하게 0.8의 유사도 값을 가진다면  
공통 평가한 아이템의 수가 10개인 사용자 더 믿을만할것입니다 즉 신뢰도가 더 높을 것 입니다.

이번에는 공통으로 평가한 아이템의 수가 일정값 이상인 사용자만 추출하여 피어그룹으로 지정해서 협업 필터링을 계산합니다.

# 데이터 읽기

무비렌즈의 유저의 정보(u.user) 읽기

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#베이스 경로 설정
base = '/content/drive/MyDrive/RecoSys/Data'

# u.user 파일 경로 설정
u_user_path = os.path.join(base, 'u.user')

#필요한 컬럼 정의
u_cols = ['user_id','age','sex','occupation','zip_code']

#데이터 읽어오기
users = pd.read_csv(u_user_path, sep='|', names = u_cols, encoding='latin-1')
#users 데이터 프레임에 인덱스(user_id) 지정
users = users.set_index('user_id')

#상위 5개
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


무비렌즈의 영화의 정보(u.item) 읽기

In [2]:
#u.item의 파일 경로 설정
u_item_path = os.path.join(base, 'u.item')

#필요한 컬럼 정의
i_cols = ['movie_id','title','release date','video release date','IMDB URL','unknown','Action',
          'Adventure','Animation','Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama',
          'Fantasy','Film-Noir','Horror','Musical', 'Mystery','Romance','Sci-Fi','Thriller','War','Western']

# 데이터 읽어오기
movies = pd.read_csv(u_item_path, sep='|',names =i_cols, encoding='latin-1')
# movies 데이터 프레임에 인덱스(movie_id) 지정
movies = movies.set_index('movie_id')

#상위 5개
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


무비렌즈의 평점 정보(u.data)읽기

In [3]:
#u.data의 파일경로 지정
u_data_path = os.path.join(base, 'u.data')

#필요한 컬럼 정의
r_cols = ['user_id', 'movie_id','rating','timestamp']

#데이터 읽어오기
ratings = pd.read_csv(u_data_path, sep='\t',names = r_cols, encoding='latin-1')

#상위 5개
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# 평가지표


In [4]:
from sklearn.metrics import mean_squared_error
def RMSE(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))

In [5]:
def score(model, neighbor_size=0):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])

  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])

  y_true = np.array(x_test['rating'])

  return RMSE(y_true, y_pred)

# 모델링

In [6]:
# 데이터 셋 만들기
x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, stratify=y)

#sparse matrix 만들기
ratings_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [7]:
# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

matrix_dummy = ratings_matrix.copy().fillna(0)

user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index=ratings_matrix.index,
                               columns=ratings_matrix.index)

#사용자의 평점 평균을 구함
ratings_mean = ratings_matrix.mean(axis=1)
#평점 - 사용자의 평점 평균을 하여 평점 편차를 구함
ratings_bias = (ratings_matrix.T - ratings_mean).T

In [17]:
# 평점에 평가가 되어 있다면 1 없다면 0인 행렬 생성
rating_binary_1 = np.array(ratings_matrix>0).astype(float)
# 위에 만든 행렬 전치(Transpose)
rating_binary_2 = rating_binary_1.T

#각 유저 ID가 평가한 영화의 수
counts = np.dot(rating_binary_1, rating_binary_2)
counts = pd.DataFrame(counts, index=ratings_matrix.index,
                      columns = ratings_matrix.index).fillna(0)

In [18]:
counts

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,204.0,8.0,3.0,4.0,39.0,53.0,83.0,15.0,5.0,41.0,...,44.0,9.0,27.0,9.0,18.0,10.0,30.0,6.0,17.0,49.0
2,8.0,46.0,7.0,2.0,1.0,18.0,10.0,4.0,3.0,10.0,...,11.0,7.0,15.0,9.0,12.0,6.0,12.0,4.0,6.0,7.0
3,3.0,7.0,40.0,6.0,0.0,5.0,8.0,6.0,1.0,5.0,...,1.0,0.0,6.0,2.0,5.0,1.0,9.0,1.0,3.0,0.0
4,4.0,2.0,6.0,18.0,1.0,1.0,8.0,4.0,1.0,2.0,...,1.0,1.0,3.0,2.0,4.0,1.0,5.0,2.0,5.0,3.0
5,39.0,1.0,0.0,1.0,131.0,17.0,68.0,14.0,3.0,19.0,...,31.0,3.0,10.0,2.0,11.0,4.0,16.0,2.0,11.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,10.0,6.0,1.0,1.0,4.0,8.0,10.0,2.0,1.0,4.0,...,4.0,14.0,12.0,6.0,19.0,37.0,4.0,4.0,1.0,11.0
940,30.0,12.0,9.0,5.0,16.0,32.0,43.0,11.0,4.0,28.0,...,27.0,6.0,13.0,8.0,10.0,4.0,80.0,3.0,11.0,16.0
941,6.0,4.0,1.0,2.0,2.0,5.0,6.0,4.0,3.0,5.0,...,3.0,3.0,10.0,5.0,6.0,4.0,3.0,17.0,2.0,3.0


대각 행렬은 1번 사용자가 평가한 영화의 수  
(1, 2)의 경우에는 사용자 1,2가 공통으로 평가한 영화의 수

In [46]:
def CF_knn_bias_sig(user_id, movie_id, neighbor_size= 0):
  if movie_id in ratings_bias:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = ratings_bias[movie_id].copy()

    no_rating = movie_ratings.isnull()
    common_counts = counts[user_id]
    low_significance = common_counts < SIG_LEVEL
    condition = (no_rating | low_significance)
    none_rating_idx = movie_ratings[condition].index

    movie_ratings = movie_ratings.drop(none_rating_idx)
    sim_scores = sim_scores.drop(none_rating_idx)
    if neighbor_size == 0:
      prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      prediction = prediction + ratings_mean[user_id]
    
    else:
      if len(sim_scores) > MIN_RATINGS:
        neighbor_size= min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
        prediction = prediction + ratings_mean[user_id]
      else:
        prediction = ratings_mean[user_id]
  else:
    prediction = ratings_mean[user_id]
  if prediction <= 1:
    prediction = 1
  elif prediction >= 5:
    prediction = 5
  
  return prediction

SIG_LEVEL = 3
MIN_RATINGS = 3
score(CF_knn_bias_sig, 30)

0.9435058279638221

사용자 평가 경향만 고려한 모델과 비교해보았을 때는 성능향상이 되지는 않았지만

SIG_LEVEL, MIN_RATINGS를 조절하면서 최적값을 찾는다면 성능을 향상시킬 수 있습니다.